<a href="https://colab.research.google.com/github/mohripan/Belajar-NLP/blob/main/NLP_and_Sequence_Model_Part_5_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip

--2023-06-11 17:46:40--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2023-06-11 17:46:41--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  4.98MB/s    in 5m 53s  

2023-06-11 17:52:34 (5.08 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]



In [3]:
!mkdir glove

In [4]:
!unzip -q /content/glove.42B.300d.zip -d glove

In [5]:
import spacy

In [26]:
nlp = spacy.load('en_core_web_sm')

sentences = ['Believe in yourself and your abilities.',
             'Never give up on your dreams, no matter how big or small they may seem.',
             'Keep going, even when things get tough.',
             'A flock of flamingos.',
             'An octopus eight arms.',
             'A popsicle on a hot day.']

labels = [1, 1, 1, 0, 0, 0]

def preprocess_text(text):
    # Use SpaCy for preprocessing
    doc = nlp(text.lower())
    # Use lemmatized version of the word unless it's a pronoun, and remove punctuation and spaces
    return [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc if token.is_alpha]

In [10]:
import numpy as np

In [11]:
def load_glove_embeddings(path):
  embeddings = {}
  with open(path, 'r', encoding = 'utf-8') as file:
    for line in file:
      values = line.strip().split()
      word = values[0]
      vector = np.array(values[1:], dtype = 'float32')
      embeddings[word] = vector
    return embeddings

glove_embeddings = load_glove_embeddings('/content/glove/glove.42B.300d.txt')

print(glove_embeddings['text'])

[ 1.5670e-01 -2.2406e-01  1.8370e-01  6.4616e-01  2.8366e-01  5.7250e-01
 -2.9115e+00  7.4529e-01  2.3426e-03  1.4693e-02 -1.1565e-01  1.7561e-01
  3.5424e-01 -2.1065e-02 -2.9127e-01 -1.9452e-01  3.1869e-01  6.2684e-01
  4.0918e-02  9.2003e-02 -1.9427e-01 -4.9586e-01  1.4694e-01 -2.7903e-01
 -7.5842e-02 -8.2243e-01  6.2999e-01 -3.9374e-02 -1.8939e-01 -4.3787e-01
  3.7667e-01  9.6839e-03  3.3840e-01  2.2171e-01 -2.6935e-01  5.0523e-02
  1.1554e-01 -5.1907e-02  4.2702e-01  3.7942e-01 -1.3508e-01  3.7566e-01
 -4.2238e-01 -2.4842e-01  4.2722e-01 -3.1349e-01  1.0448e-01 -8.6473e-02
 -7.2372e-03 -5.7965e-01 -7.2468e-02  2.8465e-01 -3.5662e-01 -1.2014e-01
  1.7577e-01  3.7206e-01 -1.6491e-01  3.1192e-01 -6.0786e-01 -1.1540e+00
  4.9371e-01 -9.9394e-02  5.0650e-01  1.2165e-01  4.8474e-02  4.3641e-01
 -7.1655e-01 -2.3858e-01 -1.6920e-01 -2.0122e-02  3.3982e-01 -2.1287e-01
 -2.1000e-02 -4.8859e-01  1.1695e-01 -1.4504e-01 -2.5419e-01 -4.2225e-02
  1.2277e-01 -4.4175e-01 -1.1966e+00 -7.9468e-01  3

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

In [13]:
class SimpleRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(SimpleRNN, self).__init__()
    
    self.hidden_size = hidden_size

    self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
    self.i2o = nn.Linear(input_size + hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim = 1)

  def forward(self, input, hidden):
    combined = torch.cat((input, hidden), 1)
    hidden = self.i2h(combined)
    output = self.i2o(combined)
    output = self.softmax(output)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, self.hidden_size)

In [33]:
n_hidden = 128
n_categories = 1
n_dimensions = glove_embeddings['text'].shape[0]

rnn = SimpleRNN(n_dimensions, n_hidden, n_categories)

In [34]:
optimizer = optim.Adam(rnn.parameters())
criterion = nn.BCEWithLogitsLoss()

In [64]:
labels = torch.tensor(labels, dtype = torch.float32).reshape(-1, 1)

<ipython-input-64-48009a22e2a9>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype = torch.float32).reshape(-1, 1)


In [69]:
num_epochs = 5

for i in range(num_epochs):
  for sentence, label in zip(sentences, labels):
    preprocessed_sentence = preprocess_text(sentence)
    sentence_vectors = []
    for word in preprocessed_sentence:
      if word in glove_embeddings:
        sentence_vectors.append(glove_embeddings[word])
    sentence_vectors = torch.tensor(sentence_vectors)
    hidden = rnn.initHidden()
    for i in range(sentence_vectors.size()[0]):
      optimizer.zero_grad()
      output, hidden = rnn(sentence_vectors[i].view(1, -1), hidden)

      label_reshaped = torch.clone(label)
      label_reshaped = label_reshaped.reshape(-1, 1)

      loss = criterion(output, label_reshaped)